In [ ]:
'''

This notebook is stil work in progress

'''

# Preprocessing

This notebook is used to process the raw transcripts into a format that can then be used for model training and testing.


In [27]:
#!/usr/bin/env/python
%matplotlib inline
import pandas as pd
import numpy as np
import glob
import string
import unicodedata
import matplotlib
import matplotlib.pyplot as plt
import re

In [28]:
#Load Subset 1 and Subset 2
path1 = '../data/Subset1_WithDialogActs/'
subdirs = glob.glob('%s*'%(path1))

path2 = '../data/Subset2_WithDialogActs/'
subdirs.extend(glob.glob('%s*'%(path2)))                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [29]:
def resolve_label(label):
    if label == '1':
        return '0'
    elif label =='2':
        return '1'
    elif label =='3':
        return '2'
    elif label =='4':
        return '3'
    elif label =='5':
        return '4'
    elif label =='8':
        return '5'
    elif label =='9':
        return '6'
    else:
        return 'nan'
    
def resolve_slabel(label):
    if label == '1':
        return '0'
    elif label =='2':
        return '1'
    elif label =='3':
        return '2'
    elif label =='4':
        return '3'
    elif label =='5':
        return '4'
    else:
        return 'nan'

In [30]:
'''
To avoid overfitting the models we carefully divide the training and testing set. 
Testing set include some teachers who particiapted in Year 1. The transcripts from these teachers are not included
in the training set. The teachers in this list were picked at random.
'''

train_subdirs =[]
test_subdirs = []
for item in subdirs:
    if item.find("Travers.Spring") != -1 or item.find("Barry.Spring") != -1 \
    or item.find("Carroll") != -1 or item.find("Saunders.Spring") != -1 \
    or item.find("Keene") != -1 or item.find("Carter") != -1 or item.find("Basker") != -1 :
        test_subdirs.append(item)
    else:
        train_subdirs.append(item)
print(len(train_subdirs), len(test_subdirs))

504 63


In [31]:
data = None
temp = []
flag = 0
for dir in subdirs: #replace with test_subdirs to generate test data
    if dir.endswith('.xlsx') and not dir.startswith('~'):
        try:
            temp_data = None
            load_data = pd.read_excel(dir, engine='openpyxl')
            #print(dir, list(load_data.columns))
            load_data['Sentence'] = load_data['Sentence'].str.replace("[\(\[].*?[\)\]]", "", regex=True)
            load_data = load_data[load_data['Sentence'] != ""]
            mask = load_data['Sentence'].str.contains('(.*[\(].*|.*[\)].*|.*[\[].*|.*[\]].*)|(^[.?!]+$)|(^.*[0-9]{1,2}:[0-9]{1,2}:[0-9]{1,2}.*$)',regex=True,na=False)
            temp_data = load_data[:][~mask]
            temp_data = temp_data.loc[:, ~temp_data.columns.str.contains('^Unnamed')]
            temp_data['Transcript'] = dir.split('/')[len(dir.split('/'))-1]
            for item in temp_data.columns:
                if item not in ['Teacher Tag', 'TimeStamp' ,'Turn','Student Tag','Sentence','Speaker','Transcript','DAMSLTag']:
                    temp_data = temp_data.drop(columns=[item])
            if flag == 0:
                data = temp_data
                flag = 1
            else:
                if len(data.columns) > 8:
                    print(data.columns,dir)
                data = data.append(temp_data, ignore_index=True,sort=False)
        except Exception as e:
            print(e,dir)
            pass 

In [32]:
len(data), len(subdirs)

(239397, 567)

In [33]:
data = data[data['Speaker'].notna()]

In [34]:
temp_df = data[data['Speaker'] == 'T']
temp_df['DAMSLTag'].value_counts(), temp_df['DAMSLTag'].value_counts().sum()


(sd    114189
 b      12128
 sv      7325
 aa      3922
 ba      3511
 %       1888
 qy       672
 fc         2
 Name: DAMSLTag, dtype: int64,
 143637)

In [6]:
data[(data['DAMSLTag'].notna()) & (data['DAMSLTag'] != '%')]

,TimeStamp,Turn,Speaker,Sentence,Student Tag,DAMSLTag,Transcript,Teacher Tag
0,NaN,1,T,"My name's Ms. Villarin, I don't know if Ms. Fe...",NaN,sd,7th grade math.xlsx_with_dialog_acts.xlsx,NaN
1,NaN,1,T,"So today, her and I will be teaching a lesson ...",NaN,sd,7th grade math.xlsx_with_dialog_acts.xlsx,NaN
2,NaN,1,T,"So you can treat me the way you treat her, if ...",NaN,sd,7th grade math.xlsx_with_dialog_acts.xlsx,NaN
3,NaN,1,T,Okay?,NaN,b,7th grade math.xlsx_with_dialog_acts.xlsx,NaN
4,NaN,1,T,So today we're going to be doing a really cool...,NaN,sd,7th grade math.xlsx_with_dialog_acts.xlsx,NaN
...,...,...,...,...,...,...,...,...
207798,NaN,290,T,"Okay, clear answers to the questions.",NaN,sd,Talkback.Year1.Strathom.Fall.121119.xlsx_with_...,1 - none
207799,NaN,290,T,I'm hoping you all rewrote clear answers to yo...,NaN,sd,Talkback.Year1.Strathom.Fall.121119.xlsx_with_...,1 - none
207800,NaN,290,T,You did accurate work.,NaN,sd,Talkback.Year1.Strathom.Fall.121119.xlsx_with_...,1 - none
207801,NaN,290,T,You made revisions and I am going to look to s...,NaN,sd,Talkback.Year1.Strathom.Fall.121119.xlsx_with_...,2 - keeping everyone together


In [6]:
data = data[(data['DAMSLTag'].notna()) & (data['DAMSLTag'] != '%')]
#print(data.head(), set(data['DAMSLTag']))
data = data[(data['Teacher Tag'].notna()) ^ (data['Student Tag'].notna())]

In [13]:
dataset = []
bad_sentences = []
bad_speakers = []
num_sentences = []
bad_entries = []
translator = str.maketrans('', '', string.punctuation)
for entry in data.iterrows():
    try:
        if type(entry[1]['Speaker']) == int or entry[1]['Speaker'] == float:
            bad_speakers.append(entry[1]['Speaker'])
            continue
        if repr(type(entry[1]['Sentence'])) == "<type 'datetime.time'>" or repr(type(entry[1]['Sentence'])) == "<type 'datetime.datetime'>" or repr(entry[1]['Speaker']) == 'nan':
            bad_sentences.append(entry[1]['Sentence'])
            continue
    
        if repr(type(entry[1]['Sentence'])) == "<type 'int'>" or repr(type(entry[1]['Sentence'])) == "<type 'float'>":
            num_sentences.append(str(entry[1]['Sentence']))
            entry[1]['Sentence'] = str(entry[1]['Sentence']).translate(translator)
        if repr(type(entry[1]['Sentence'])) == "<type 'str'>":
            entry[1]['Sentence'] = str(entry[1]['Sentence']).translate(translator)
            
        temp = unicodedata.normalize('NFKD', str(entry[1]['Sentence']).translate(translator))
        
        # Turn, Speaker, Sentence, Tag
        try:
            tag = resolve_label(str(entry[1]['Teacher Tag']).split('-')[0].strip())
        except Exception as e:
            print(e,'####')
        try:
            stag = resolve_slabel(str(entry[1]['Student Tag']).split('-')[0].strip())
        except Exception as e:
            print(e,'####')
        
        sent = ' '.join([re.sub(r"[^a-zA-Z0-9]+", ' ', k) for k in temp.split(" ")])
        
        if str(stag) != 'nan':
            #print([entry[1]['Turn'],'S',sent.strip(),tag,stag,entry[1]['Transcript']])
            dataset.append([entry[1]['Turn'],'S',sent.strip(),tag,stag,entry[1]['DAMSLTag'],entry[1]['Transcript']])
        elif str(tag) != 'nan':
            #print([entry[1]['Turn'],'T',sent.strip(),tag,stag,entry[1]['Transcript']])
            dataset.append([entry[1]['Turn'],'T',sent.strip(),tag,stag,entry[1]['DAMSLTag'],entry[1]['Transcript']])
       
    except Exception as e:
        print(e)
        bad_entries.append([entry[1]['Sentence'], entry[1]['Speaker']])
        continue

In [14]:
df = pd.DataFrame({'Turn':[i[0] for i in dataset], 'Speaker':[i[1] for i in dataset], \
                   'Sentence':[i[2] for i in dataset], 'Tag':[i[3] for i in dataset],\
                   'StudentTag':[i[4] for i in dataset], 'DAMSLTag':[i[5] for i in dataset],
                   'Transcript':[i[6] for i in dataset]})
df = df.applymap(lambda x: x.encode('unicode_escape').decode('utf-8') if isinstance(x, str) else x)
df.head(), set(df['Tag']), set(df['StudentTag'])

(  Turn Speaker                                           Sentence   Tag  \
 0  NaN       T                Do you want to say it to the camera     1   
 1  NaN       S                          No I don t want to say it  None   
 2  NaN       S  This is going to be the number in the square m...  None   
 3  NaN       T  Yeah what have you written here that s very in...     0   
 4  NaN       S                Zero times zero plus one equals one  None   
 
   StudentTag DAMSLTag                                         Transcript  
 0        nan       sd  Video Mosaic Grade 6 2 variables 14.xlsx_with_...  
 1          0       sd  Video Mosaic Grade 6 2 variables 14.xlsx_with_...  
 2          3       sd  Video Mosaic Grade 6 2 variables 14.xlsx_with_...  
 3        nan       sd  Video Mosaic Grade 6 2 variables 14.xlsx_with_...  
 4          4       sd  Video Mosaic Grade 6 2 variables 14.xlsx_with_...  ,
 {'0', '1', '2', '3', '4', '5', '6', None, 'nan'},
 {'0', '1', '2', '3', '4', 'nan'})

In [23]:
for item in sorted(df['Transcript'].unique()):
    print(item, len(df[df['Transcript']==item]))

7th grade math.xlsx_with_dialog_acts.xlsx 142
Boats and Fish 1_Grade 4.xlsx_with_dialog_acts.xlsx 210
Boats and Fish 3_Grade 4.xlsx_with_dialog_acts.xlsx 113
Boats and Fish 4_Grade 4 .xlsx_with_dialog_acts.xlsx 187
Boats and fish 2_Grade 4.xlsx_with_dialog_acts.xlsx 112
Borko.TranscriptofWholeClassDebrief.xlsx_with_dialog_acts.xlsx 60
CECILIO DIMAS pt 1.xlsx_with_dialog_acts.xlsx 87
CECILIO DIMAS pt 2.xlsx_with_dialog_acts.xlsx 44
CECILIO DIMAS pt 3.xlsx_with_dialog_acts.xlsx 62
Comparing Fractions 1_Grade 4.xlsx_with_dialog_acts.xlsx 80
Comparing Fractions 2_Grade 4.xlsx_with_dialog_acts.xlsx 179
Comparing Fractions 3_Grade 4.xlsx_with_dialog_acts.xlsx 107
Comparing Fractions 4_Grade 4.xlsx_with_dialog_acts.xlsx 94
Comparing Fractions 5_Grade 4.xlsx_with_dialog_acts.xlsx 67
Fraction as Number 1_Grade 4.xlsx_with_dialog_acts.xlsx 70
Fraction as Number 2_Grade 4.xlsx_with_dialog_acts.xlsx 108
Fraction as Number 3_Grade 4.xlsx_with_dialog_acts.xlsx 97
Fraction as Number 4_Grade 4.xlsx_wi

TalkBack.Year1.Davies.Spring.031120.xlsx_with_dialog_acts.xlsx 352
TalkBack.Year1.Gautier.Fall.100419.xlsx_with_dialog_acts.xlsx 518
TalkBack.Year1.Gautier.Fall.100819.xlsx_with_dialog_acts.xlsx 374
TalkBack.Year1.Gautier.Fall.120319-1.xlsx_with_dialog_acts.xlsx 554
TalkBack.Year1.Gautier.Fall.120319-2.xlsx_with_dialog_acts.xlsx 454
TalkBack.Year1.Gautier.Fall.121219.xlsx_with_dialog_acts.xlsx 717
TalkBack.Year1.Gautier.Spring.010920.xlsx_with_dialog_acts.xlsx 552
TalkBack.Year1.Gautier.Spring.011420-1.xlsx_with_dialog_acts.xlsx 607
TalkBack.Year1.Gautier.Spring.011420-2.xlsx_with_dialog_acts.xlsx 681
TalkBack.Year1.Gautier.Spring.031020.xlsx_with_dialog_acts.xlsx 390
TalkBack.Year1.Gibson.Fall.120619.xlsx_with_dialog_acts.xlsx 721
TalkBack.Year1.Gibson.Fall.121019.xlsx_with_dialog_acts.xlsx 506
TalkBack.Year1.Gibson.Fall.121219.xlsx_with_dialog_acts.xlsx 421
TalkBack.Year1.Gibson.Fall.121319.xlsx_with_dialog_acts.xlsx 34
TalkBack.Year1.Gibson.Fall.121619.xlsx_with_dialog_acts.xlsx 367

TalkBack.Year1.Travers.Spring.020620-2.xlsx_with_dialog_acts.xlsx 810
TalkBack.Year1.Travers.Spring.020620-3.xlsx_with_dialog_acts.xlsx 558
TalkBack.Year1.Travers.Spring.022120-1.xlsx_with_dialog_acts.xlsx 797
TalkBack.Year1.Travers.Spring.022120-2.xlsx_with_dialog_acts.xlsx 947
TalkBack.Year1.Travers.Spring.022620.xlsx_with_dialog_acts.xlsx 532
TalkBack.Year1.Travers.Spring.030520.xlsx_with_dialog_acts.xlsx 689
TalkBack.Year1.Wolfram.Spring.010920-1.xlsx_with_dialog_acts.xlsx 293
TalkBack.Year1.Wolfram.Spring.010920-2.xlsx_with_dialog_acts.xlsx 574
TalkBack.Year1.Wolfram.Spring.011420-1.xlsx_with_dialog_acts.xlsx 463
TalkBack.Year1.Wolfram.Spring.011420-2.xlsx_with_dialog_acts.xlsx 297
TalkBack.Year1.Wolfram.Spring.011520.xlsx_with_dialog_acts.xlsx 107
TalkBack.Year1.Wolfram.Spring.012120-1.xlsx_with_dialog_acts.xlsx 299
TalkBack.Year1.Wolfram.Spring.012120-2.xlsx_with_dialog_acts.xlsx 564
TalkBack.Year1.Wolfram.Spring.013120.xlsx_with_dialog_acts.xlsx 459
TalkBack.Year1.Wolfram.Sprin

TalkBack.Year2.Smith.Spring.042821.xlsx_with_dialog_acts.xlsx 381
TalkBack.Year2.Smith.Spring.050421.xlsx_with_dialog_acts.xlsx 483
TalkBack.Year2.Smith.Spring.081321.xlsx_with_dialog_acts.xlsx 298
TalkBack.Year2.Smith.Spring.102320.xlsx_with_dialog_acts.xlsx 389
TalkBack.Year2.Strathom.Fall.102020.xlsx_with_dialog_acts.xlsx 892
TalkBack.Year2.Strathom.Fall.102220.xlsx_with_dialog_acts.xlsx 291
TalkBack.Year2.Strathom.Fall.102320.xlsx_with_dialog_acts.xlsx 796
TalkBack.Year2.Strathom.Fall.103020.xlsx_with_dialog_acts.xlsx 538
TalkBack.Year2.Strathom.Fall.112020.xlsx_with_dialog_acts.xlsx 530
TalkBack.Year2.Strathom.Fall.120220.xlsx_with_dialog_acts.xlsx 621
TalkBack.Year2.Strathom.Fall.120420.xlsx_with_dialog_acts.xlsx 471
TalkBack.Year2.Strathom.Fall.121020.xlsx_with_dialog_acts.xlsx 991
TalkBack.Year2.Strathom.Spring.010821.xlsx_with_dialog_acts.xlsx 322
TalkBack.Year2.Strathom.Spring.011321-1.xlsx_with_dialog_acts.xlsx 523
TalkBack.Year2.Strathom.Spring.011321-2.xlsx_with_dialog_act

In [16]:
df['DAMSLTag'].value_counts()

sd    181939
b      25071
sv     10062
aa      7610
ba      5030
%       4189
qy      1066
fc        10
Name: DAMSLTag, dtype: int64

In [ ]:
temp_df = df[df['Speaker'] == 'S']
temp_df['StudentTag'].value_counts(), temp_df['Tag'].value_counts().sum()

In [9]:
writer = pd.ExcelWriter('../data/train_data_503_dailog_acts.xlsx') #replace with test_data_63 for test data
df.to_excel(writer)
writer.save()

In [ ]:
'''
Pre-processing 

- All lower cased
- All punctuation removed
- Converted all teachers to T tags
- Converted all students to S tags
- T labels range from  [0-6] - 0 stands for None
- S are labelled None
- Compiled all the data into single excel sheet
- At present there are 91891 sentences in total

Cleaning

- [End of Class], [Class Ends] - Sentence in brackets
- If Sentence contains a timestamp or date time stamp
- If sentence is just . or ! or ? or empty
- Speaker is a number (All of them are nans)
- Speaker is nan


'''

In [10]:
#Prepare Tuples from data - Student
data = pd.read_excel('../data/train_data_503_dailog_acts.xlsx', engine='openpyxl') #replace with test_data_63 to generate test data
empty_previous = ''
sentence_tuple = []
y = []
flag  = 0
# T->T 0->1 T->S S->S S->T
for row in data.iterrows():
    if row[1]['Speaker'] == 'T' and flag == 0:
        temp =  row[1]['Sentence']
        flag = 1
        continue
    if row[1]['Speaker'] == 'S' and flag == 0:
        temp =  row[1]['Sentence']
        flag=1
        sentence_tuple.append([empty_previous,temp,row[1]['DAMSLTag'],row[1]['StudentTag']])
        y.append(row[1]['StudentTag'])
        continue
    if row[1]['Speaker'] == 'T' and flag == 1:
        temp =  row[1]['Sentence']
        continue
    if row[1]['Speaker'] == 'S' and flag == 1:
        sentence_tuple.append([temp,row[1]['Sentence'],row[1]['DAMSLTag'], row[1]['StudentTag']])
        y.append(row[1]['StudentTag'])
        temp =  row[1]['Sentence']
        continue
        
df = pd.DataFrame({'text_a':[i[0] for i in sentence_tuple], \
                   'text_b':[i[1] for i in sentence_tuple], \
                   'damsl_labels':[i[2] for i in sentence_tuple],
                   'labels':[i[3] for i in sentence_tuple]})

print(len(df),set(y))
df.to_csv('../data/train_student_dact.tsv',sep='\t',index=False)   #replace with test_student to generate test data  

42857 {0.0, 1.0, 2.0, 3.0, 4.0}


In [11]:
#Prepare Tuples from data - teacher
data = pd.read_excel('../data/train_data_503_dailog_acts.xlsx', engine='openpyxl') #replace with test_data_63 to generate test data
empty_student = ''
sentence_tuple = []
y = []
flag  = 0
# T->T 0->1 T->S S->S S->T
for row in data.iterrows():
    if row[1]['Speaker'] == 'S' and flag == 2:
        temp =  row[1]['Sentence']
        continue
    if row[1]['Speaker'] == 'S' and flag == 1:
        temp =  row[1]['Sentence']
        flag = 2
        continue
    if row[1]['Speaker'] == 'T' and flag == 0:
        temp =  row[1]['Sentence']
        if repr(row[1]['Tag']) == 'nan':
            print(row[1])
        flag = 1
        sentence_tuple.append([empty_student,temp,row[1]['DAMSLTag'],row[1]['Tag']])
        y.append(row[1]['Tag'])
        continue
    if row[1]['Speaker'] == 'T' and flag == 1:
        temp =  row[1]['Sentence']
        if repr(row[1]['Tag']) == 'nan':
            print(row[1])
        flag = 1
        sentence_tuple.append([empty_student,temp,row[1]['DAMSLTag'],row[1]['Tag']])
        y.append(row[1]['Tag'])
        continue
    if row[1]['Speaker'] == 'T' and flag == 2:
        if repr(row[1]['Tag']) == 'nan':
            print(row[1])
        sentence_tuple.append([temp,row[1]['Sentence'],row[1]['DAMSLTag'],row[1]['Tag']])
        y.append(row[1]['Tag'])
        flag = 1
        continue

df = pd.DataFrame({'text_a':[i[0] for i in sentence_tuple],\
                   'text_b':[i[1] for i in sentence_tuple],\
                   'damsl_labels':[i[2] for i in sentence_tuple],\
                   'labels':[i[3] for i in sentence_tuple]})

print(len(df),set(y))
df.to_csv('../data/train_teacher_dact.tsv',sep='\t',index=False)  #replace with test_teacher to generate test data 

143829 {0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0}
